In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
pretrain_features = {
    "learning_rate": 0.1,
    "batch_size": 256,
    "epochs": 1000,
    "eval_size": 4*256,
    "momentum": 0.005,
    "weight_decay": 0.0001,
}

In [21]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    scaler = StandardScaler()
    x_pretrain = scaler.fit_transform(x_pretrain)
    x_train = scaler.transform(x_train)
    x_test_transed = scaler.transform(x_test)
    x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [22]:
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()

c:\Users\linan\.conda\envs\DL\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [23]:
class AE(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.01),
            nn.BatchNorm1d(1000),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.01),
            nn.BatchNorm1d(1000),
            nn.Linear(1000, 1000),
            nn.BatchNorm1d(1000))

        self.decoder = nn.Sequential(
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.01),
            nn.BatchNorm1d(1000),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.01),
            nn.BatchNorm1d(1000),
            nn.Linear(1000, 1000)
            )
            
        for m in self.modules():
            if isinstance(m, nn.Linear):    
                nn.init.xavier_uniform_(m.weight)
        
        
    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def encode(self, x):
        return self.encoder(x)

In [24]:
eval_size = pretrain_features["eval_size"]
batch_size = pretrain_features["batch_size"]
ae_model = AE()
ae_model.train()
ae_model.to(device)

def train_autoencoder():
    x_tr, x_val, y_tr, y_val = train_test_split(x_pretrain, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    # optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_features['learning_rate'], weight_decay=pretrain_features['weight_decay'])
    optimizer = torch.optim.SGD(ae_model.parameters(), lr=pretrain_features['learning_rate'], momentum=pretrain_features['momentum'], weight_decay=pretrain_features['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = pretrain_features['epochs']
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, _] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, _] in val_loader:
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_autoencoder()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: train loss: 1.4051969674863445, val loss: 1.1628666818141937
Epoch 2: train loss: 0.991415945932936, val loss: 1.0502719581127167
Epoch 3: train loss: 0.9317291708445868, val loss: 1.0165606439113617
Epoch 4: train loss: 0.9062194476054408, val loss: 0.9983953535556793
Epoch 5: train loss: 0.8856553137789206, val loss: 0.9777085185050964
Epoch 6: train loss: 0.8663969801283086, val loss: 0.958357185125351
Epoch 7: train loss: 0.8476379026349407, val loss: 0.9405882358551025
Epoch 8: train loss: 0.8293201430332268, val loss: 0.9255602359771729
Epoch 9: train loss: 0.8117932967016488, val loss: 0.9102109670639038
Epoch 10: train loss: 0.7948995638384379, val loss: 0.8917213380336761
Epoch 11: train loss: 0.778454465887119, val loss: 0.8790036141872406
Epoch 12: train loss: 0.7625459319078545, val loss: 0.8614891320466995
Epoch 13: train loss: 0.7477716118126514, val loss: 0.8473160117864609
Epoch 14: train loss: 0.7332883733076739, val loss: 0.8343290388584137
Epoch 15: train lo

In [25]:
class Net(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
        # and then used to extract features from the training and test data.
        self.seq = nn.Sequential(
            nn.Linear(1000, 40),
            nn.LeakyReLU(0.01),
            nn.BatchNorm1d(40),
            nn.Linear(40, 1)
        )

        for m in self.modules():
            if isinstance(m, nn.Linear):    
                nn.init.xavier_uniform_(m.weight)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        # TODO: Implement the forward pass of the model, in accordance with the architecture 
        # defined in the constructor.
        x = self.seq(x)
        return x


In [26]:
featured_x_train = ae_model.encoder(torch.tensor(x_train, dtype=torch.float).to(device))

In [27]:
def get_regression_model(X, y):

    class Model(nn.Module):
            """
            The model class, which defines our feature extractor used in pretraining.
            """
            def __init__(self):
                """
                The constructor of the model.
                """
                super().__init__()
                # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
                # and then used to extract features from the training and test data.
                self.seq = nn.Sequential(
                    nn.Linear(1000, 100),
                    nn.LeakyReLU(0.01),
                    nn.BatchNorm1d(100),
                    nn.Linear(100, 1)
                )

                for m in self.modules():
                    if isinstance(m, nn.Linear):    
                        nn.init.xavier_uniform_(m.weight)

            def forward(self, x):
                """
                The forward pass of the model.

                input: x: torch.Tensor, the input to the model

                output: x: torch.Tensor, the output of the model
                """
                # TODO: Implement the forward pass of the model, in accordance with the architecture 
                # defined in the constructor.
                x = self.seq(x)
                return x
    
    """
    This function returns the regression model used in the pipeline.

    input: None

    output: model: sklearn compatible model, the regression model
    """
    # TODO: Implement the regression model. It should be able to be trained on the features extracted
    # by the feature extractor.
    model = Net()
    model.to(device)
    model.train()

    x_tr, x_val, y_tr, y_val = train_test_split(X, y, test_size=10, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
    

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.4, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 10000
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, y] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            loss_tr = loss.item()
        for [x, y] in val_loader:
            x = x.to(device)
            y = y.to(device)
            predictions = model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss_val = loss.item()
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-8):
            print(f"Early stop at epoch {epoch+1}")
            break


    return model

one_model = get_regression_model(featured_x_train, y_train)

C:\Users\linan\AppData\Local\Temp\ipykernel_18016\784483399.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)


  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 1: train loss: 5.1358819007873535, val loss: 6.634143829345703
Epoch 2: train loss: 9.775274276733398, val loss: 3.2889747619628906
Epoch 3: train loss: 5.008784294128418, val loss: 3.526519536972046
Epoch 4: train loss: 3.6139533519744873, val loss: 3.464247226715088
Epoch 5: train loss: 2.789952278137207, val loss: 3.165626287460327
Epoch 6: train loss: 2.292762041091919, val loss: 2.820711851119995
Epoch 7: train loss: 2.0040178298950195, val loss: 2.513446807861328
Epoch 8: train loss: 1.82310950756073, val loss: 2.2671782970428467
Epoch 9: train loss: 1.6883230209350586, val loss: 2.079256057739258
Epoch 10: train loss: 1.570068359375, val loss: 1.9184671640396118
Epoch 11: train loss: 1.453373908996582, val loss: 1.7668430805206299
Epoch 12: train loss: 1.333536148071289, val loss: 1.624039888381958
Epoch 13: train loss: 1.210031270980835, val loss: 1.479267954826355
Epoch 14: train loss: 1.0839898586273193, val loss: 1.3588405847549438
Epoch 15: train loss: 0.9586302042007

In [28]:
result_file = "results-ae.csv"

In [29]:
y_pred = np.zeros(x_test.shape[0])
y_pred = one_model(ae_model.encoder(torch.tensor(x_test.to_numpy(), dtype=torch.float).to(device))).squeeze(-1).detach().cpu().numpy()

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print(f"Predictions saved to {result_file}, all done!")

Predictions saved to results-ae.csv, all done!
